In [1]:
import os
import json
import time
import openai
import pandas as pd
from tqdm import tqdm

In [2]:
with open('openai_key.txt', encoding='utf-8') as f:
    openai_key = f.readlines()[0]

In [3]:
openai.api_key = openai_key

In [4]:
def get_completion(intial_setup, prompt , model="gpt-3.5-turbo", temperature=0):
  
  messages = [{"role": "system", "content": intial_setup}]
  messages.append({"role": "user", "content": prompt})

  response = openai.ChatCompletion.create(model=model,
                                          messages=messages,
                                          temperature=temperature) # this is the degree of randomness of the model's output
  if response.choices:
    return response.choices[0].message.content
  else:
    return None

In [5]:
with open('initial_prompt_v2.json') as f:
    initial_prompt = json.load(f)

In [6]:
prompt = "Annie must like Starbucks because all white girls like Starbucks."
get_completion(initial_prompt, prompt)

"TITLE: Stereotyping is not a valid argument\nFACT1: Stereotyping is a form of prejudice that assumes all members of a group share the same characteristics or behaviors. It is not a valid argument because it relies on assumptions rather than evidence.\nMYTH: All white girls like Starbucks.\nFALLACY: This argument is an example of a hasty generalization fallacy, which occurs when a conclusion is drawn based on insufficient evidence. It assumes that all white girls have the same preferences and behaviors, which is not true. Additionally, even if many white girls do like Starbucks, it does not mean that all of them do.\nFACT2: Let's not make assumptions about people based on their race or gender. It is important to recognize and appreciate individual differences and avoid making sweeping generalizations."

In [7]:
df = pd.read_csv('climatedataset2.csv')

In [9]:
for i in tqdm(df.index):
    
    attempts = 0
    prompt = df.loc[i, 'fact_checked_segment']

    while attempts < 3:

        try:
            debunk = get_completion(initial_prompt, prompt)
            if debunk is None:
                attempts += 1
                continue

            row = df.loc[i].to_dict()
            row['debunk'] = debunk
            json.dump(row, open(f"outputs2/{i}.json", 'w'))
            break

        except Exception as e:
            print(f'{i} row failed attempt {attempts}: {e}')
            attempts += 1
            
    time.sleep(1)

  2%|▏         | 12/685 [03:18<3:45:32, 20.11s/it]

12 row failed attempt 0: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID ecfc96fd5201f3a7d7cb3aa05ed6e417 in your message.)


  3%|▎         | 18/685 [05:35<3:21:08, 18.09s/it]

18 row failed attempt 0: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7eb3d9a99c4caca567d5f649d0850661 in your message.)


  4%|▍         | 29/685 [09:37<2:53:17, 15.85s/it]

...

 48%|████▊     | 330/685 [1:35:18<2:20:31, 23.75s/it]

330 row failed attempt 0: We could not parse the JSON body of your request. (HINT: This likely means you aren't using your HTTP library correctly. The OpenAI API expects a JSON payload, but what was sent was not valid JSON. If you have trouble figuring out how to fix this, please contact us through our help center at help.openai.com.)
330 row failed attempt 1: We could not parse the JSON body of your request. (HINT: This likely means you aren't using your HTTP library correctly. The OpenAI API expects a JSON payload, but what was sent was not valid JSON. If you have trouble figuring out how to fix this, please contact us through our help center at help.openai.com.)
330 row failed attempt 2: We could not parse the JSON body of your request. (HINT: This likely means you aren't using your HTTP library correctly. The OpenAI API expects a JSON payload, but what was sent was not valid JSON. If you have trouble figuring out how to fix this, please contact us through our help center at help.op

 52%|█████▏    | 358/685 [1:42:31<1:41:15, 18.58s/it]

358 row failed attempt 0: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 9ca2ad5e0d29aadb46dc2a5b0cdaa242 in your message.)


100%|██████████| 685/685 [3:18:44<00:00, 17.41s/it]  


In [12]:
files = [int(x.split('.')[0]) for x in os.listdir('outputs2/')]
files = sorted(files)
review = [x for x in df.index if x not in files]
review

[73, 330]

In [13]:
df.loc[73]

fact_checked_segment              NaN
comment_by_fact-checker           NaN
split                             dev
label                      no fallacy
Name: 73, dtype: object

In [14]:
df.loc[330]

fact_checked_segment                                                     NaN
comment_by_fact-checker    This is only true if you selectively pick stud...
split                                                                  train
label                                                             no fallacy
Name: 330, dtype: object

In [16]:
debunkings = []
for file_name in os.listdir('outputs2/'):
    if file_name.endswith('.json') and not file_name.startswith('.') and file_name != "all.json" and file_name != "all.csv":
        print(file_name)
        _debunkings = json.load(open('outputs2/'+file_name, 'r'))
        debunkings.append(_debunkings)

json.dump(debunkings, open('outputs2/all.json', 'w'))
debunkings_df = pd.read_json('outputs2/all.json')
debunkings_df.to_csv('outputs2/all.csv', index=False, encoding='utf-8')

683.json
379.json
396.json
115.json
545.json
400.json
284.json
512.json
142.json
457.json
292.json
338.json
154.json
504.json
441.json
380.json
553.json
103.json
416.json
251.json
601.json
314.json
197.json
528.json
482.json
178.json
20.json
656.json
206.json
343.json
590.json
98.json
77.json
210.json
640.json
355.json
586.json
569.json
139.json
61.json
617.json
247.json
302.json
181.json
36.json
494.json
549.json
119.json
41.json
375.json
230.json
660.json
16.json
288.json
322.json
637.json
267.json
508.json
158.json
334.json
271.json
621.json
57.json
363.json
676.json
226.json
461.json
174.json
524.json
318.json
94.json
436.json
573.json
123.json
420.json
82.json
6.json
135.json
565.json
359.json
477.json
532.json
498.json
162.json
476.json
533.json
499.json
163.json
358.json
421.json
7.json
83.json
134.json
564.json
95.json
437.json
572.json
122.json
319.json
460.json
175.json
525.json
362.json
677.json
227.json
56.json
335.json
270.json
620.json
509.json
159.json
289.json
323.json


In [17]:
debunkings

[{'fact_checked_segment': '“Recent computer forecasts suggest that if greenhouse gas emissions continue at a high level, parts of Antarctica could break up rapidly, causing the ocean to rise six feet or more by the end of this century. That is double the maximum increase that an international climate panel projected only four years ago. But those computer forecasts were described as crude even by the researchers who created them. ‘We could be decades too fast, or decades too slow,’said one of them, Robert M. DeConto of the University of Massachusetts, Amherst. ‘There are still some really big question marks about the trajectory of future climate around Antarctica.’”',
  'comment_by_fact-checker': ' The “recent computer forecasts” referenced here likely refer to the numerical model developed by Rob DeConto1. Different from other ice-sheet models it includes the so-called “Marine Ice Cliff Instability”, which at present is not widely accepted in the ice-sheet modeling community. Includin